In [3]:
!pip install python-telegram-bot google-generativeai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 11.3 MB/s eta 0:00:00


In [3]:
!pip install aiogram google-generativeai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.6/666.6 kB 12.3 MB/s eta 0:00:00


In [5]:
import os
import textwrap
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import google.generativeai as genai

# Конфигурация
TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Инициализация Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Технический промпт
SYSTEM_PROMPT = textwrap.dedent("""
    Ты - профессиональный AI-ассистент технической поддержки.
    Отвечай точно и по делу, используя техническую терминологию.
    Если не знаешь ответа - так и скажи.
    Форматируй ответы с использованием markdown, выделяя ключевые моменты.
    """)

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /start"""
    await update.message.reply_text(
        "🚀 Привет! Я технический ассистент с интеграцией Gemini AI.\n"
        "Задайте мне любой технический вопрос!"
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /help"""
    help_text = (
        "📝 Доступные команды:\n"
        "/start - начать работу\n"
        "/help - получить справку\n"
        "/clear - очистить историю диалога\n\n"
        "Просто напишите ваш технический вопрос, и я постараюсь помочь!"
    )
    await update.message.reply_text(help_text)

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик текстовых сообщений"""
    try:
        user_input = update.message.text

        # Создаем полный промпт с системными инструкциями
        full_prompt = f"{SYSTEM_PROMPT}\n\nВопрос пользователя: {user_input}"

        # Генерируем ответ
        response = model.generate_content(full_prompt)

        # Форматируем ответ
        formatted_response = f"🤖 **Ответ:**\n{response.text}"

        await update.message.reply_markdown(formatted_response)

    except Exception as e:
        error_message = f"⚠️ Произошла ошибка: {str(e)}"
        await update.message.reply_text(error_message)

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик ошибок"""
    print(f"Ошибка: {context.error}")
    await update.message.reply_text("😞 Произошла ошибка при обработке запроса")

if __name__ == '__main__':
    # Инициализация приложения
    app = Application.builder().token(TELEGRAM_TOKEN).build()

    # Регистрация обработчиков
    app.add_handler(CommandHandler("start", start_command))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Обработчик ошибок
    app.add_error_handler(error_handler)

    # Запуск бота
    print("🚀 Бот запущен...")
    app.run_polling()

InvalidToken: You must pass the token you received from https://t.me/Botfather!